# 1 prepare the data first

 1) download the movie in USA data json
 2) collect the useful information from the movies.json
 3) cover it into the vectore using SentenceTransformer

In [1]:
!wget https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json

--2023-04-28 19:51:30--  https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25818279 (25M) [text/plain]
Saving to: ‘movies.json’

movies.json         100%[===================>]  24,62M  6,17MB/s    in 4,1s    

2023-04-28 19:51:35 (6,08 MB/s) - ‘movies.json’ saved [25818279/25818279]



In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

Notice that there is some useless information in the original JSON file. So only keep the important information by pre-prepare data.  
After pre-preparation, save new JSON and load it into Pandas DataFrame.

In [3]:
with open('movies.json', 'r') as f:
    dict_ori = json.load(f)

#pre-prepare the data
def pre_process(dict):
    pre_proce_dict= []
    filtered_list = [item for item in dict if 'cast' in item and len(item['cast']) > 0 and 'extract' in item and len(item['extract']) > 0 ]
    keys_to_keep = ['title', 'year', 'cast', 'extract']
    for d in filtered_list:
        filtered_dict = {k: d[k] for k in keys_to_keep if k in d}
        pre_proce_dict.append(filtered_dict)
    return pre_proce_dict
    
dict_01 = pre_process(dict_ori)

# Convert list of dictionaries to JSON string with indentation
json_string = json.dumps(dict_01, indent=2)

# Write JSON string to file and save it into data folder.
with open('data/useful_movies.json', 'w') as f:
    f.write(json_string)

# Load JSON string into a Pandas DataFrame
df_movie = pd.read_json(json_string)
df_movie

,title,year,cast,extract
0,Feeding Sea Lions,1900,[Paul Boyton],Feeding Sea Lions is short silent film featuri...
1,Alice in Wonderland,1903,[May Clark],Alice in Wonderland is a 1903 British silent f...
2,The Automobile Thieves,1906,"[J. Stuart Blackton, Florence Lawrence]",The Automobile Thieves is an American crime-dr...
3,Humorous Phases of Funny Faces,1906,[J. Stuart Blackton],Humorous Phases of Funny Faces is a 1906 short...
4,Ben Hur,1907,[William S. Hart],Ben Hur is a 1907 American silent drama film s...
...,...,...,...,...
33923,Wonka,2023,"[Timothée Chalamet, Keegan-Michael Key, Sally ...",Wonka is an upcoming musical fantasy film dire...
33924,Aquaman and the Lost Kingdom,2023,"[Jason Momoa, Amber Heard, Willem Dafoe, Patri...",Aquaman and the Lost Kingdom is an upcoming Am...
33925,Untitled Ghostbusters: Afterlife sequel,2023,"[Mckenna Grace, Carrie Coon, Finn Wolfhard, Pa...",The untitled Ghostbusters: Afterlife sequel is...
33926,Rebel Moon,2023,"[Sofia Boutella, Charlie Hunnam, Ray Fisher, D...",Rebel Moon is an upcoming American epic space ...


Then use the available model (SentenceTransformer) to convert the important information into vectors.  
Here in the movie data frame, the 'extract' is the most important element which can summary a movie more accurately.

In [74]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device="cpu") 

I tried to get the vector locally but it spent over 40 minites to finish.  
Due to the limitation of my PC, the conversion was applied in Colab to finish, and download the data vector(vectors_movies.npy) for later to use.

In [75]:
vectors_1 = []
batch_size_1 = 64
batch_1 = []
for row in tqdm(df_movie.itertuples()):
  description = row.extract
  batch_1.append(description)
  if len(batch_1) >= batch_size_1:
    vectors_1.append(model.encode(batch_1))
    batch_1 = []

if len(batch_1) > 0:
  vectors_1.append(model.encode(batch_1))
  batch_1 = []

vectors_1 = np.concatenate(vectors_1)
np.save('data/vectors_movies.npy', vectors_1, allow_pickle=False)

0it [00:00, ?it/s]

# 2 prepare the vector search engine

 - Using docker to active Qdrant
 - active int_vector_search.py to collect and upload the data into Qdrant

# 3 Use FastAPI as a recommedation API  
- define how to give the recommedation in NeuralSearcher.py
- service.py

### The rest part of code was for testing.

In [18]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams
import os

In [19]:
import numpy as np
import json
import os

In [25]:
CODE_DIR = os.getcwd()
ROOT_DIR = os.path.dirname(CODE_DIR)
DATA_DIR = os.path.join(CODE_DIR, 'data')

COLLECTION_NAME = "movies"

QDRANT_HOST = os.environ.get("QDRANT_HOST", "localhost")
QDRANT_PORT = os.environ.get("QDRANT_PORT", 6333)

In [36]:
vectors_path = os.path.join(DATA_DIR, 'vectors_movies.npy')
vectors = np.load(vectors_path)
vector_size = vectors.shape[1]

payload_path = os.path.join(DATA_DIR, 'useful_movies.json')
with open(payload_path, 'r') as f:
    data_dict = json.load(f)

payload_01 = []
for idx, doc in enumerate(data_dict):
  payload_01.append(data_dict)

In [41]:
qdrant_client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

qdrant_client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=vector_size, distance="Cosine")
)

True

In [42]:
BATCH_SIZE = 256

qdrant_client.upload_collection(
    collection_name=COLLECTION_NAME,
    vectors=vectors,
    payload=data_dict,
    ids=None,
    batch_size=BATCH_SIZE,
    parallel=2
)